### Loading AG News with Torchtext

The AG News dataset is one of many included Torchtext.
It can be found grouped together with many of the other text classification datasets.
While we can download the source text online, Torchtext makes it retrievable with a quick API call&ast;. If you are running this notebook on your machine,  you can uncomment and run this block:

In [7]:
import torchtext
import torch
from torchtext.datasets.ag_news import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

The first step is to build a vocabulary with the raw training dataset. Here we use built in factory function build_vocab_from_iterator which accepts iterator that yield list or iterator of tokens. Users can also pass any special symbols to be added to the vocabulary.

In [2]:
train_iter, test_iter = AG_NEWS(split=("train", "test"))
flag, text = next(iter(train_iter)) #It each iterator has the category/value and text
tokenizer = get_tokenizer('basic_english') # It retrieves the tool which allows you to encode the text
print(text)
print(tokenizer(text))

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']


In [3]:
def yield_tokens(data_iter): #Converts every text into an array of tokens, it returns it in an iterable way
    for _, text in data_iter: 
        yield tokenizer(text)
        
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"]) #It generates the vocab, which represents the words into indexes
vocab.set_default_index(vocab["<unk>"])

> What is yield in Python? [link](https://www.simplilearn.com/tutorials/python-tutorial/yield-in-python#:~:text=let's%20get%20started.-,What%20Is%20Yield%20In%20Python%3F,of%20simply%20returning%20a%20value)<br><br>The Yield keyword in Python is similar to a return statement used for returning values or objects in Python. However, there is a slight difference. The **yield statement returns a generator object to the one who calls the function which contains yield**, instead of simply returning a value. 

In [4]:
print(vocab(['here', 'is', 'an', 'example']))

[475, 21, 30, 5297]


In [5]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [6]:
print(text_pipeline('here is the an example'))
print(label_pipeline('10'))

[475, 21, 2, 30, 5297]
9


In [8]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

TypeError: 'DataLoader' object is not subscriptable